<a id="top"></a> 

# Basic Visualizations and Descriptive Statistics of Spike Train Data

<div class="question">
    
_**Synopsis**_ 

**Data:** Spontaneous spiking activity from a retinal neuron in culture, exposed to low-light and high-light environments.


**Goal:** Visualize spike trains, compute and interpret descriptive statistics, and build simple models of interspike interval distributions as a function of the ambient light level.


**Tools:** Raster plots, interspike interval histograms, firing rate, autocorrelograms, maximum likelihood estimation, Kolmogorov-Smirnov plots.
</div>

* [Introduction](#.)
* [Data analysis](#data-analysis)
    1. [Visual inspection](#visual-inspection)
    2. [Examining the Interspike Intervals](#isi)
    3. [Examining Binned Spike Increments](#bsi)
    4. [Computing autocorrelations for the Increments](#autocorrelations)
    5. [Computing Autocorrelations of the ISIs](#acISI)
    6. [Building Statistical Models of the ISIs](#models)
* [Summary](#summary)
* [Appendix: Spike Count Mean and Variance for a Poisson Process](#appendix)

## Introduction

Neurons in the retina typically respond to patterns of light displayed over small sections of the visual field. However, when retinal neurons are grown in culture and held under constant light and environmental conditions, they will still spontaneously fire action potentials. In a fully functioning retina, this spontaneous activity is sometimes described as background firing activity, which is modulated as a function of visual stimuli. It is useful to understand the properties of this background activity in order to determine in future experiments how these firing properties are affected by specific stimuli.

### Case Study Data

A researcher examining the background firing properties of one of these neurons contacts us to discuss his data. He runs two experiments in which he records the spiking activity in two different states, one with the room lights off (low ambient light levels) and the other with the room lights on (high ambient light levels). He would like to collaborate with us to determine whether there is a difference in background firing between these two conditions, and whether one environment is more conducive to future experimental analyses. He records the spiking activity as spike timestamps for 30 seconds in each condition.

### Goal

Typically the first step in any data analysis involves visualizing and using simple descriptive statistics to characterize pertinent features of the data. For time series data that take on a continuous value at each time point, like the field potentials analyzed in earlier chapters, we typically start by simply plotting each data value as a function of time. For spike train data, things can become a bit more complicated. One reason for this is that there are multiple equivalent ways to describe the same spike train data. The data could be stored as a sequence of spike times; as a sequence of waiting times between spikes, or interspike intervals; or as a discrete time series indicating the number of spikes in discrete time bins. Knowing how to manipulate and visualize spike train data using all these different representations is the first step to understanding the structure present in the data and is the primary goal of this chapter.

### Tools

We develop tools in this chapter to visualize spike train data and to provide basic statistical methods appropriate for analyzing spike trains.

<a id="data-analysis"></a>
       
## Data Analysis

<a id="visual-inspection"></a>

Our data analysis begins with loading and visualizing the data.

In [ ]:
# Set-up

# import required packages
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline

# set default figure size and with tight layout
rcParams['figure.figsize'] = (12,3)
rcParams['figure.autolayout'] = True

# load data
data = sio.loadmat('Ch8-spikes-1.mat')  # Load the ECoG data

<div class="question">

**Q.** How can we extract the variables of interest from `data`? (*Hint*: Recall that the `loadmat()` function outputs a dict.)

</div>

In [ ]:
# since we know `data` is a dict, let's look at the keys
data.keys()

There are two non-private keys in `data`:

`SpikesLow`: spike times over 30 s in the low ambient light condition

`SpikesHigh`: spike times over 30 s in the high ambient light condition

**Q.** Let's assign these keys as vectors to the the variables `ts_dark` and `ts_light` so that we can work with them directly.

In [ ]:
# Two different ways to flatten a numpy array to 1-d. The first only works for 2-d arrays
ts_dark = data['SpikesLow'][0]
ts_light = data['SpikesHigh'].reshape(-1)

Each variable is a single vector that gives a set of increasing spike times (aka timestamps, or 'ts') for the associated condition. The two vectors are of different sizes because the neuron fired a different number of spikes in each condition.

<div class="question">
    
**Q.** How do `len`, `shape`, and `size` differ in returning information about ndarrays?

</div>

In [ ]:
# The standard library function `len`, returns an int of the length of the first dimension
print(len(ts_dark))
# Consider what would have happened had we not flattened our vector first:
print(len(data['SpikesLow']))

In [ ]:
# The `shape` attribute is a tuple of the number of elements in each dimension
print(ts_dark.shape)

In [ ]:
# The `size` attribute is an int of the total number of elements in the array
print(ts_dark.size)
ts_dark.size == data['SpikesLow'].size == 750

Inspection of the sizes of the vectors `ts_dark` and `ts_light` reveals an important fact: the neuron fires more in the light condition. To make this observation more concrete, let’s compute the firing rate ($f$), defined mathematically as

$$
f = \frac{n}{T},
\tag{1}
$$

where $n$ is the number of spikes over the time interval $T$.

<div class="question">
    
**Q.** What is the firing rate $f$ of the neuron recorded in the two conditions?
    
</div>

In [ ]:
# We know that we have 30 seconds worth of data
T = 30
n_light = ts_light.size
n_dark = ts_dark.size
f_light = n_light / T
f_dark = n_dark / T

print('f_light =', f_light)
print('f_dark =', f_dark)

These calculations allow us to compute a simple number representative of one aspect of the data: the firing rate over the entire duration of the recording. Do the two datasets exhibit a statistically significant change in the firing structure between conditions? Or, does the difference in firing rates lie within the range of expected fluctuations between any two trials of random spiking data? To answer these types of questions, we need to develop statistical methods that are appropriate for analyzing spike trains. Let’s first look at the data more carefully and visualize the structure of the spiking in the low ambient light condition.

<div class="question">

**Q.** How can we visualize the data as a rastergram? Create a figure and axes using `plt.subplots`, and plot the spike times for the dark condition vs. a vector of zeros and the spike times for the light condition vs. a vector of ones on the same axes. Apply appropriate labels to the axes.
    
(*Hints*: Use `np.zeros` or `np.zeros_like` and `np.ones` or `np.ones_like` to create a vector of zeros and ones. Use the axes `plot` method (and appropriately set the marker and linestyle) to plot the timestamps vs the vectors of zeros or ones.)

</div>

In [ ]:
# Create a figure, and plot raster from each condition

fig1, ax1 = plt.subplots()
# Plot ts vs. zeros for dark condition (*Note, it is good practice to set optional
# input args via keywords, like with `marker='.'` and `linestyle=''` below)
ax1.plot(ts_dark, np.zeros_like(ts_dark), marker='.', linestyle='')
# Plot ts vs. ones for light condition
ax1.plot(ts_light, np.ones_like(ts_light), marker='.', linestyle='')
ax1.set_xlabel('Time (s)')
# zoom out on y axis
ax1.set_ylim([-1, 2])
# and hide y ticks
ax1.set_yticks([])
# add legend
ax1.legend(['Dark Condition', 'Light Condition'])
# add title
ax1.set_title('Spike Rasters for Same Neuron Across Light Conditions')

With the data visualized in this way, we’re now able to ask an interesting question: What structure do we notice in the two spike trains? At first glance, our answer might be “not much.” Spikes occur fairly regularly throughout the 30 s interval under both conditions. At this scale, it may even look like there more spikes in the dark than in the light condition, even though we know the opposite is true.

The spike times themselves do not seem to be directly comparable between these conditions. Often, when we examine data from a stimulus response experiment, we expect to see regions where spiking activity increases or decreases as a function of a *changing* stimulus. In this case, the stimulus (the ambient light level) remains constant over both experiments, respectively. How else can we analyze these data and identify differences in the spiking activity (if any) between the two conditions?

[Back to top](#top)


<a id="isi"></a>
### Examining the Interspike Intervals

So far, we have examined the long-term structure of the spiking. Let’s now focus on a shorter-term structure that occurs within a single second or less.

In [ ]:
# Plot rasters to show one second of data for both conditions from T = 10-11 s

fig2, ax2 = plt.subplots()
# set plot labels for dark and light conditions
ax2.plot(ts_dark, np.zeros_like(ts_dark), marker='.', linestyle='', label='Dark Condition')
ax2.plot(ts_light, np.ones_like(ts_light), marker='.', linestyle='', label='Light Condition')
ax2.set_xlim([10, 11])              
ax2.set_ylim([-1, 2])
ax2.set_yticks([])
ax2.set_xlabel('Time [s]')
# since we set labels for our plots, we can call the `legend` method without
# specifying the input args, and it will use the labels
ax2.legend()

The code above was essentially copied and pasted from the previous section with a couple minor updates to change the time interval and the way we create the legend. It is generally poor practice to repeat a lot of similar calculations in such a manner, and if we needed to do so, we should create a function instead.

<div class="question">

**Q.**: Create a function, `plot_rasters`, that takes as inputs an ndarray matrix of spikes, `spks` (where each row is a different spike train and each column represents a spike timestamp), and a time range, `t`, as an ndarray vector, and displays rasters for each spike train in `spks`, returning the figure, `fig`, axes, `ax`, and rastergrams, `rasters`, which the function creates. 
    
(*Hint*: most of the code above can be slightly adjusted to be used in the function body)

</div>

In [ ]:
def plot_rasters(spks, t):
    """ 
    Plots rasters for each row of spike trains in `spks` within the time interval `t`
    
    Parameters:
    -----------
        spks (ndarray): a 2-d matrix of spike trains (# of spike trains X # of timestamps) 
        t (ndarray | list | tuple): two numbers specifying a time range
    
    Returns:
    --------
        fig(figure): a handle to the figure that holds the created rasters
        ax(axes): a handle to the plot that holds the created rasters
        rasters(lines): a handle to the rasters plotted on `ax`
    """
    
    # get the number of spike trains
    n_trains = len(spks)
    # create the figure
    fig, ax = plt.subplots()
    # plot the spike trains using a comprehension
    rasters = [ax.plot(spks[train], np.zeros_like(spks[train])+train, marker='.', linestyle='') \
               for train in range(0,n_trains)]
    # set axes xlim according to `t`
    ax.set_xlim([t[0], t[1]])
    # set axes ylim to zoom out
    ax.set_ylim([-1, n_trains+1])
    ax.set_yticks([])
    ax.set_xlabel('Time (s)')
    return fig, ax, rasters

<div class="question">

**Q:** After creating `plot_rasters`, run it on `ts_dark` and `ts_light` for the interval between 10-11 seconds, and create a test using `assert` to show this code executes as expected.

(*Hints:* Vertically concatenate `ts_dark` and `ts_light` into a single ndarray after padding the shorter one with nans (see `np.pad`, `np.nan`, and `np.vstack`). 
    
</div>

In [ ]:
# nan pad the `ts_dark` vector to match lengths with `ts_light`

ts_dark_padded = np.pad(ts_dark, pad_width = (0, (len(ts_light)-len(ts_dark))), \
                        mode = 'constant', constant_values = np.NaN)

# Concatenate vectors vertically

# (We have to use double parentheses because `np.vstack` only takes a single ndarray-like
# argument, so here we combine 'ts_dark_padded' and `ts_light` into a tuple - we could have
# also combined them into a list. This is a common theme for many numpy operations on ndarrays)
spks = np.vstack((ts_dark_padded, ts_light))

# Run function

t = [10,11]
fig, ax, rasters = plot_rasters(spks,t)

# Run test

# get individual rasters for dark and light condition
r_dark = rasters[0][0]
r_light = rasters[1][0]

# assert that the number of points plotted matches the number of timestamps for each condition
assert r_light.get_ydata().size == ts_light.size and r_dark.get_ydata().size == ts_light.size

# note that `r_dark.get_ydata().size == ts_light.size` because we ran the function 
# on `ts_dark_padded`, not `ts_dark`

<div class="question">

**Q:** Create a function, `plot_rasters2`, that takes as inputs an arbitrary number of individual spike train vectors (e.g. `[train1, train2,...trainN]`), and a time range, `t`, equivalent to `t` in `plot_rasters`. `plot_rasters2` should return the same outputs as `plot_rasters`. 

(*Hint:* we can use `plot_rasters` as a template and replace the `spks` input arg with `*args` as an input arg to gather the arbitrary number of individual spike train vectors. Note, when using `*args` to gather an arbitrary number of input args, `*args` must come last in the function definition)
    
</div>

In [ ]:
def plot_rasters2(t, *args):
    """ 
    Plots rasters for each spike train timestamp vector in `*args*` within the time interval `t`
    
    Parameters:
    -----------
        *args (ndarray | list | tuple): spike train timestamp vectors with length == # of timestamps 
        t (ndarray | list | tuple): two numbers specifying a time range
    
    Returns:
    --------
        fig(figure): a handle to the figure that holds the created rasters
        ax(axes): a handle to the plot that holds the created rasters
        rasters(lines): a handle to the rasters plotted on `ax`
    """
    
    # create the figure
    fig, ax = plt.subplots()
    # plot the spike trains using a comprehension
    rasters = [ax.plot(args[a], np.zeros_like(args[a])+a, marker='.', linestyle='') \
               for a in range(0, len(args))]
    # set axes xlim according to `t`
    ax.set_xlim([t[0], t[1]])
    # set axes ylim to zoom out
    ax.set_ylim([-1, len(args)+1])
    ax.set_yticks([])
    ax.set_xlabel('Time (s)')
    return fig, ax, rasters

<div class="question">

**Q:** After creating `plot_rasters2`, run it on `ts_dark` and `ts_light` for the interval between 10-11 seconds, and create a test to show this code executes as expected.
    
</div>

In [ ]:
# Run function

t = [10,11]
fig, ax, rasters = plot_rasters2(t, ts_dark, ts_light)

# Run test

# get individual rasters for dark and light condition
r_dark = rasters[0][0]
r_light = rasters[1][0]

# assert that the number of points plotted matches the number of timestamps for each condition
assert r_light.get_ydata().size == ts_light.size and r_dark.get_ydata().size == ts_dark.size

# note that now `r_dark.get_ydata().size == ts_dark.size` (as opposed to the case in 
# `plot_rasters`) because we didn't have to pad `ts_dark` for `plot_rasters2`

Inspecting smaller time intervals, we might notice bursts of spikes that cluster near each other in time, interspersed with longer periods that contain less spiking. These patterns of bursts and quiescence look different between the dark and light conditions. Visual inspection is an important tool, but we would like a quantitative result. How might we compare this fine temporal structure in the two conditions?

One approach to further characterizing the differences in spiking between the two conditions is to transform the data. One of the most useful transformations focuses on the waiting times between the spikes, or interspike intervals (ISIs), instead of the spike times themselves. 

<div class="question">

**Q:** Compute the ISIs for the two conditions by using `np.diff`.

</div>

In [ ]:
isi_dark = np.diff(ts_dark)    # Compute ISIs in the low-light condition
isi_light = np.diff(ts_light)  # Compute ISIs in the high-light condition

`np.diff` takes the difference between each pair of consecutive elements in its input argument ndarray, which is exactly what we want to compute the ISIs.

<div class="question">

**Q:** For a first visualization of the ISIs, plot the ISIs for both conditions. Create a 1x2 subplot via `plt.subplots`, where `isi_dark` is plotted on the left subplot in blue, and `isi_light` is plotted on the right subplot in red. Share the x and y axes for both figures, and set the axes ylim to be the maximum ISI time for either condition. Appropriately label both subplots.

(*Hints*: Use `np.max` on the concatenated array of `isi_light` and `isi_dark` to get the ylim. Use the axes `plot` method on the ISIs to create one plot for each axes.)
    
</div>

In [ ]:
# Plot the ISIs between each pair of spikes for both conditions:

# create 2 subplots with shared axes and increase the figure size
fig3, ax3 = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(12,6))
ax3[0].plot(isi_dark, color='b')
ax3[1].plot(isi_light, color='r')
ax3[0].set_xlabel('ISI Number')
ax3[0].set_ylabel('Time (s)')
# get `ylim` as max ISI for either condition
ylim = np.max(np.hstack((isi_dark, isi_light)))
# increase the yticks to show every 0.05s from 0-`ylim`
ax3[1].set_yticks(np.arange(0,ylim+0.05,0.05))
# set the axes ylim from 0-`ylim`
ax3[1].set_ylim([0, ax3[1].get_yticks()[-1]])
# set titles
ax3[0].set_title('ISIs Dark Condition')
ax3[1].set_title('ISIs Light Condition')

We see that the ISI values range from small times (less than 0.05 s) to large times (over 0.65 s). In this way, the visualization provides some insight into how the ISI values differ across the conditions.

Plots of the ISI vectors provide some information about the data (e.g., the approximate range of the ISI values), but there’s more insight to be gained. To that end, let’s now implement another approach to visualizing these types of data: the histogram. 

<div class="question">

**Q:** Plot histograms for both sets of ISIs, again using 2 subplots with the color and axes scheme we used above. Use 1 ms bins from 0-100 ms. Appropriately label both subplots.

(*Hints*: Use `np.arange` to set the bins (in ms). Use the axes `hist` method on the ISIs (converted to ms) and bins to create one histogram for each axes.)
    
</div>

In [ ]:
# Plot the ISI histograms for both conditions:

# create 2 subplots with shared axes and increase the figure size
fig4, ax4 = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(12,6))
# set the bins in ms
bins = np.arange(0, 100, 1)
# create the histograms in ms
ax4[0].hist(isi_dark*1000, bins, color='b')
ax4[1].hist(isi_light*1000, bins, color='r')
ax4[0].set_xlabel('Time (ms)')
ax4[0].set_ylabel('Counts')
ax4[0].set_title('ISIs Dark Condition')
ax4[1].set_title('ISIs Light Condition')

In the ISI histogram of the dark condition, very few counts occur at small ISI values (near 0 ms) and high ISI values (beyond approximately 100 ms). Instead, the distribution of counts is broadly peaked in the approximate interval 5–20 ms.

What does the ISI distribution reveal about the spiking activity in the dark condition? We could conclude from the ISI distribution that many spikes are separated by time intervals 5–20 ms. So, we might be tempted to imagine near-periodic spiking with a period 5–20 ms:

<a id="fig:8.6a"></a>

![Cartoon representation of spiking activity.](imgs/8-6a.png "Near-periodic spiking, with period 5&mdash;20 ms.")

However, the histogram contains additional structure beyond the single broad peak. Indeed, the histogram has a long tail. Therefore, the intervals between spikes are varied. We often see ISIs in the 5–20 ms interval, but we also find much longer ISIs. The structure of the ISI histogram is consistent with bursting activity, which consists of intervals of rapid spiking interspersed with quiescence:

<a id="fig:8.6b"></a>

![Cartoon representation of spiking activity.](imgs/8-6b.png "Bursting activity, with both short and long intervals between spikes.")

The intervals of rapid spiking produce many shorter ISIs, and the longer intervals produce (typically fewer) longer ISIs. We may conceptualize a bursting neuron as having two time scales: fast and slow. From the shape of the histogram of the dark condition, we could conclude that the spike train data in the dark condition are more consistent with bursting activity than with periodic, metronome-like spiking activity. We could conclude that this is also the case in the light condition. We could claim that the neuron exhibits bursting activity in both conditions, but clearly there are some difference bewtween the two ISI histograms: consider the impact of the large proportion of small ISIs in the light condition. We'll continue on to examine this further.
    


</div>

[Back to top](#top)
<a id="bsi"></a>
### Examining Binned Spike Increments

Another common approach to analyzing spiking data is to discretize time into bins of fixed width and count the number of events that occur in each time bin. This differs from the ISI histogram above, because here we are not looking at the time between events, but the number of events themselves. The sequence of spike counts across the bins is sometimes called the *increment process* for a spike train. When the time bins are sufficiently small, say, 1 ms for typical spike train data, the resulting increment process is just a sequence of zeros and ones. In this case, the time bins are so small that the probability of more than one spike occurring in each bin is virtually zero. Each tiny time bin then contains a spike (and we assign that bin a value of 1) or does not (and we assign that bin a value of 0). This idea of representing the spike train as a sequence of zeros and ones for small bin increments will be important when we build statistical models of the spike trains in a [later chapter](../09/). In this section, we compute the increment process with multiple bin sizes in order to characterize the amount of variability in the spiking data and to examine temporal dependencies between spikes.

<div class="question">

**Q:** Compute and plot the increment process for the entire spike trains for both conditions in time bins of 50 ms, again using 2 subplots with the color and axes scheme we used above. Appropriately label both subplots. 
    
(*Hints*: Use `np.arange` to set the bins. Use `np.histogram` to compute the increment process from the spike times and the bins. Use the axes `plot` method on the computed bins and increments to create one plot for each axes, and appropriately set the marker and linestyle as was done for the rastergrams.)
    
</div>

In [ ]:
# Compute the ms converted timestamps for both conditions
ts_dark_ms = ts_dark * 1000
ts_light_ms = ts_light * 1000

# set the bins in ms
bin_length = 50 # in ms
bins = np.arange(0, T*1000, bin_length)

# compute the increments (only return the first output argument, the histogram. 
# We don't need the second, the bin edges)
increment_dark, _ = np.histogram(ts_dark_ms, bins)
increment_light, _ = np.histogram(ts_light_ms, bins)

# plot the increments
fig5, ax5 = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(12,6))
# exclude the last element in `bins` when plotting, so the number of elements
# matches the number of elements of the increments
ax5[0].plot(bins[:-1], increment_dark, color='b', marker='.', linestyle='')
ax5[1].plot(bins[:-1], increment_light, color='r', marker='.', linestyle='')
ax5[0].set_xlabel('Time (ms)')
ax5[0].set_ylabel('Counts')
ax5[0].set_title('Increment Process Dark Condition')
ax5[1].set_title('Increment Process Light Condition')

One question that arises quite often is how variable these binned counts are. To illustrate this variability, let’s consider two scenarios. In the first, consider a neuron that fires perfectly regularly, like a metronome. In this case, we expect the number of spikes in each time bin to be nearly identical. On the other hand, consider the scenario of a neuron that fires in irregular bursts. In this case, we expect much more variability in the number of spikes in each time bin, depending on whether a time bin contained a burst of spikes or a quiet period. To characterize this variability, a standard measure to compute is the sample Fano factor (FF). It’s easy to define the Fano factor: *FF is the sample variance of the increment process divided by the sample mean of the increment process*.

<div class="question">

**Q:** Calculate the FF from the increments computed above for both conditions, and appropriately assign the results to the variables `ff_dark_50` and `ff_light_50`, respectively.
    
</div>

In [ ]:
ff_dark_50 = increment_dark.var() / increment_dark.mean()
ff_light_50 = increment_light.var() / increment_light.mean()
print('ff_dark_50 =', ff_dark_50)

To interpret this FF, we need to introduce the concept of a Poisson process. A Poisson process is a model for a spiking process for which each spike occurrence is independent of every other spike occurrence. In other words, the probability of a neuron spiking at any instant does not depend on when the neuron fired (or did not fire) previously.

The Poisson process is rarely an accurate model for spike train data. Our biological knowledge reveals that the occurrence of a spike *does* depend on the occurrence of previous spikes (e.g., when we take into account the refractory period and phenomenon like bursting). However, the Poisson process has many nice theoretical properties, that make it a good model against which to compare the data. For example, for any Poisson process, the number of spikes in any time interval has a Poisson probability distribution for which the theoretical variance and mean are equal (see the [appendix](#appendix) at the end of the chapter). Therefore, the theoretical Fano factor for a Poisson process is exactly equal to 1.

When measuring the variability of the increments of a spike train, we typically compare it to the variability of a Poisson process. If we compute a Fano factor well below the value 1 for a particular set of increments, this suggests that the spiking is more regular than a Poisson process for the time scale at which the increments were binned. In this case, spiking activity in the past is influencing the neuron to spike in a more predictable manner in subsequent bins. If we compute a Fano factor well above the value 1, this suggests that the spiking is more variable than a Poisson process for the time scale at which the increments were binned.

Based on this and our calculations of the FF for both conditions, we could conclude that the spiking in the dark condition is more regular, and the spiking in the light condition more variable, than a Poisson process.

#### Does the observed Fano factor differ from 1?

The preceding results are somewhat unsatisfying. We claimed that in the low-light condition, the calculated Fano factor of 0.72 was well below 1. What if, instead, we calculated a Fano factor of 0.8; is that well below 1? Is a Fano factor of 0.9 well below 1? These questions highlight an important issue when drawing a conclusion from a Fano factor calculation: How far above or below the value of 1 does the calculated Fano factor have to be before we are confident that there is really a statistically significant difference in the variability from a Poisson process? After all, even if we had spiking from a true Poisson process, from one experiment to the next we would expect to find different values for the increments, and values for the sample Fano factor that fluctuate slightly above and below 1. Fortunately, a bit of statistical theory can help us out. It can be shown that the distribution of Fano factors that we might compute from a Poisson process follows a gamma distribution with shape parameter $(N - 1)/2$ and scale parameter
$2/(N - 1)$, where $N$ is the number of time bins used in the Fano factor calculation [[Eden & Kramer, 2010](https://doi.org/10.1016/j.jneumeth.2010.04.012)].




<div class="question">
    
**Q.** What is the correct value of $N$ for the increment process computed above and saved in the variable IncrementsLow50?
  
**A.** We find in Python that 

    IncrementsLow50.shape
        
is 600. Therefore, N = 600. This makes sense; 600 time bins of 50 ms duration fit between 0 s and 30 s.

</div>

With the value of $N = 600$ now determined, let’s plot the gamma distribution and investigate its shape:

In [ ]:
from scipy.stats import gamma   # Import the gamma object from the SciPy stats toolbox
N = IncrementsLow50.shape[0];   # Determine number of time bins.
shape = (N - 1) / 2             # Define the shape parameter of the gamma function
scale = 2 / (N - 1)             # ... and the scale parameter 
FF = np.linspace(.5, 1.5, 1000) # Define possible FF values
Y = gamma.pdf(FF, shape, 
              scale=scale)      # ... compute gamma distribution,
plot(FF,Y);                     # ... and plot it
xlabel('Fano Factor')           # ... with axes labeled
ylabel('Probability Density')  
show()

Notice that we’re evaluating the function `gamma.pdf()`, which returns as output the gamma probability density function. We provide three inputs to this function. The first specifies a range of Fano factors to investigate (here we choose Fano factors ranging from 0.5 to 1.5; we choose a large number of Fano factor values to make a smooth plot of the gamma distribution). The second and third inputs to the function specify the shape and scale parameters of the gamma distribution.

When $N$ is large, as it is here, the gamma distribution looks like a normal distribution (i.e., like a bell-shaped curve). We can use this distribution to construct an interval where we would expect the Fano factor to lie if the data were generated by a Poisson process. More specifically, if the data were generated by a Poisson process, then we would expect the Fano factor to lie in the 95% confidence interval around the value of 1. Let’s construct this 95% confidence interval:

In [ ]:
gamma.ppf([.025, .975], shape, scale=scale)

In this case, we use the function `gamma.ppf()` to compute the gamma inverse cumulative distribution function (or percentiles). The first input specifies the boundaries of the probabilities of interest, which range from 2.5% to 97.5%, to encompass 95% of the probability mass around the value of 1. The next two inputs specify the shape and scale parameters of the gamma distribution. Evaluating this line of code, we find the interval (0.890, 1.116). Therefore, if we observed a true Poisson process with N = 600 bins and computed the Fano factor, we would not be surprised to find values between 0.890 and 1.116. However, the observed Fano factor for an increment process with 50 ms bins in the low-light condition is well outside of this range; this result suggests that it is very unlikely that these data were generated by a Poisson process. Instead, it’s more likely that these data were generated by a process with less variability in the low-light condition (FF = 0.72).

<div class="question">


    
**Q.** Consider the Fano factor for the high-light condition. Do you believe these data were generated by a Poisson process? Why, or why not?




    
**A.** We found for the high-light condition a Fano factor of 1.78 (again using an increment process with 50 ms bins). This calculated value is well above the interval expected for a Poisson process. We therefore reject the hypothesis that these data were generated by a Poisson process. Instead, we observe more variability in the high-light condition than expected for a Poisson process (with $N = 600$).



</div>

<div class="question">

**Q.** How do the results for the Fano factor change in each condition with different choices for the bin size of the increment process (e.g., 25 ms, 100 ms, 500 ms)? Note that by changing the bin size, you also change $N$. *Challenge*: See if you can write a function that takes the data and desired binwidth as input and returns the Fano factor and 95% confidence interval.

</div>

[Back to top](#top)
<a id="autocorrelations"></a>
### Computing Autocorrelations for the Increments

Another way to characterize the history dependence structure of a spike train is with the *autocorrelation* function of the increments. A correlation coefficient describes the degree of linear dependence between any two variables. The value of the correlation ranges from -1 to 1. A correlation value of -1 indicates a perfect linear relation between the two variables with a negative slope. A value of 0 indicates no linear relation between the two variables. And a value of 1 indicates a perfect linear relation between the two variables with a positive slope. Any other value indicates that one variable can be predicted using a linear function of the other, but that prediction will be imperfect; the closer the value is to $\pm$1, the better the prediction will be. The sign of the coefficient indicates the slope of the linear relation. The following figure shows scatterplots for a variety of possible relations between two variables, and the values of the correlation coefficients.

![Correlation values for example relations between two variables.](imgs/8-9.png "Variables are plotted, one against the other, on the x-axis and y-axis. Numbers indicate values of the correlation between the two variables.")

Mathematically, the formula for the sample autocorrelation at a lag $L$ is
<a id="eq:5"></a>

$$
\rho_{xx}[L] = \frac{
\sum_{i=1}^{N - L}
(x_i - \overline x)(x_{i+L} - \overline x)
}
{
\sum_{i=1}^{N}(x_i - \overline x)^2
}
\tag{5}
$$

where $x_i$ is the $i^{th}$ data point, and $\overline x$ is the sample mean of the data over index $i$.

<div class="question">


    
**Q.** This formula is rather complicated, so let's consider a simple case: $L = 0$. What is $\rho_{xx}[0]$ at lag 0?




    
**A.** To answer this, let's substitute $L=0$ into the mathematical expression fo rthe autcorrelation,


\begin{eqnarray}
\rho_{xx}[0] &=& 
\bigg ( \sum_{i=1}^{N-0} (x_i - \overline x)(x_{i+0} - \overline x) \bigg ) 
\bigg / 
\bigg ( \sum_{i=1}^N (x_i - \overline x)^2 \bigg ) 
\\
&=&
\bigg ( \sum_{i=1}^{N} (x_i - \overline x)(x_{i} - \overline x) \bigg ) 
\bigg / 
\bigg ( \sum_{i=1}^N (x_i - \overline x)^2 \bigg ) 
\\
&=&
\bigg ( \sum_{i=1}^N (x_i - \overline x)^2 \bigg ) 
\bigg / 
\bigg ( \sum_{i=1}^N (x_i - \overline x)^2 \bigg ) 
\\
&=& 1.
\end{eqnarray}

At $L=0$, the autocorrelation is by definition equal to 1. In words, an individual dataset $x$ is perfectly correlated with itself at zero lag.



</div>

To gain some intuition for the autocorrelation, let’s consider the following relatively simple increment process:

<a id="fig:8-10a"></a>

![Example data for visual inspection of autocorrelation of an increment process.](imgs/8-10a.png "Data x as a function of time (arbitrary units).")

Our goal is to understand the autocorrelation of these data, labeled $x$. From visual inspection of the figure, we conclude that the data $x$ have mean 0, so that $\overline x = 0$; note that the data appear to oscillate between equal and opposite values over time. For simplicity, let’s focus on the numerator of equation (<a href="#eq:5" class="thumb">5<span><img src="imgs/eq5.png"></span></a>). At lag zero (i.e., $L = 0$), the numerator of (<a href="#eq:5" class="thumb">5<span><img src="imgs/eq5.png"></span></a>) tells us to multiply the data by themselves at each time index, and then sum the product. To visualize this multiplication and sum, consider multiplying the data in figure above by the following data at each index in time and then summing the result.

<a id="fig:8-10b"></a>

![Example data for visual inspection of autocorrelation of an increment process.](imgs/8-10b.png "Data x at lag L=0.")

<div class="question">


    
**Q.** What is the value of this sum?




    
**A.** We can’t answer this question without knowing the values of $x$. However, we can deduce the sign and relative size of the sum. At a time index where $x$ is positive, the product is positive; and at a time index where $x$ is negative, the product is still positive. So, the product at each time index will always be positive or zero. Therefore, we expect the sum to be a positive number. And if we add many terms, we expect this sum to be large. So, we may conclude that the value of this sum will be a large, positive number. Note that this number is the numerator of the autocorrelation. At $L = 0$, the numerator equals the denominator in (<a href="#eq:5" class="fig">5<span><img src="imgs/eq5.png"></span></a>) and the autocorrelation at lag zero is 1.



</div>

Let’s consider the numerator of the autocorrelation (<a href="#eq:5" class="fig">5<span><img src="imgs/eq5.png"></span></a>) for a small positive lag (i.e., $L > 0$). We can think of the small positive lag as shifting the data $x$ a little bit to the right:

<a id="fig:8-10c"></a>

![Example data for visual inspection of autocorrelation of an increment process.](imgs/8-10a.png "Data x.")

![Example data for visual inspection of autocorrelation of an increment process.](imgs/8-10c.png "Data x at lag L>0.")

Now, to compute the numerator of the autocorrelation, we multiply $x$ by a shifted version of $x$ at each time index and then sum the result. In this case, we find some indices where the product of $x_i$ and $x_{i+L}$ is positive, and some indices where the product of $x_i$ and $x_{i+L}$ is negative; an example of an index where the product is negative is indicated above by the black circle. Visual comparison of figures two plots suggests that the product will be positive more often then negative. Therefore, we expect the sum of these products to still be positive, although not as large as we found for the $L = 0$ case. You may have noticed some points where the data do not overlap, at the far right and far left of the figure. At these locations, the product is zero.

<div class="question">

**Q.**  Consider the autocorrelation of the data $x$ at the positive lag $L_1$ shown here:

![Example data for visual inspection of autocorrelation of an increment process.](imgs/8-10d.png "Data x at lag L1>L.")

What is the sign (positive or negative) and relative size of the numerator of the autocorrelation at this lag?

</div>

Let’s return to the spike train data of interest here, recorded in the low- and high-light conditions. For the corresponding spike train increments, the autocorrelation at a particular lag describes the relation between the spike counts in different bins separated by that lag. The autocorrelation function describes the autocorrelation across a range of lags over which we are interested. Given our visualizations of the ISI histograms (<a href="#fig:8-5" class="fig">figure<span><img src="imgs/8-5.png"></span></a>), we might expect relations between spiking events to extend up to 200 ms.

Let’s compute the autocorrelation for increment processes deduced from the spike train data in the low-light condition. We compute the autocorrelation of the 50 ms increment process for lags ranging from 0 to 200 ms. We need only three lags to cover this range; lag 1 covers 50–100 ms, lag 2 covers 100–150 ms, and lag 3 covers 150–200 ms. We can define a function to compute the autocorrelation using the function `correlate()` from the NumPy module:

In [ ]:
def autocorr(x, lags):
    xcorr = np.correlate(x - x.mean(), x - x.mean(), 'full')  # Compute the autocorrelation
    xcorr = xcorr[xcorr.size//2:] / xcorr.max()               # Convert to correlation coefficients
    return xcorr[:lags+1]                                     # Return only requested lags
    
autocorr(IncrementsLow50, 3)

Our new function `autocorr()` takes two inputs. The first input is the data for which we want to compute the autocorrelation, in this case, the increment process for the low-light condition with 50 ms time bins. The second input is the number of lags to compute. Within the function, we scale the raw autocorrelation so that we instead see correlation coefficients, which range from -1 to 1. Notice that we subtract from `IncrementsLow50` the mean of this variable before computing the autocorrelation. This command outputs a vector with four numerical values corresponding to the autocorrelation at lag indices 0 through 3. The autocorrelation at negative lags is mathematically identical to the autocorrelation at the equivalent positive lags, which is why we exclude them in the third line of the function definition.

<div class="question">


    
**Q.** Examine the numerical values returned by `autocorr()`. What do you find?




    
**A.** As expected, the autocorrelation at zero lag is exactly equal to 1; the data matches itself at lag 0. At lag 1, corresponding to 50–100 ms, the autocorrelation value is 0.04. This positive correlation value indicates that when the number of spikes in one bin is higher than expected, the number of spikes in the next bin tends to be higher than expected. Similarly, when the number of spikes in one bin is lower than expected, the number of spikes in the next bin will also tend to be lower than expected. This effect is small, however, since the autocorrelation is near zero. At lag 2, corresponding to 100–150 ms, the autocorrelation value of 0.07 is again small and positive. At lag 3, corresponding to 150–200 ms, the autocorrelation value remains small and positive at a value of 0.04.



</div>

<div class="question">


    
**Q.** How do we know whether these autocorrelation values are statistically significant?




    
**A.** This can be a difficult question when $N$ is small (less than 30), but for larger $N$ we can approximate a confidence interval about the correlation coefficient using a normal approximation with standard deviation $1/\sqrt N$. In this case, any correlation value exceeding $\pm2/\sqrt N$ is unlikely to be generated by chance and likely reflects real dependence structure. For the increment process considered here, $N = 599$, and the significance bound is $\pm 0.08$. We conclude that none of the lags has significant autocorrelation.



</div>

If we are particularly interested in the fine-scale temporal dependence structure of the spikes, we would do better to compute the autocorrelation function for more finely binned intervals. To that end, let’s repeat the autocorrelation analysis for an increment process that uses 1 ms bins. We first compute a new increment process and then apply the `autocorr()` function to this process.

In [ ]:
time_bins = np.arange(0, 30, 0.001)                    # Define the time bins
IncrementsLow1, _ = np.histogram(SpikesLow, time_bins) # ... compute the histogram to create increment process
ACFLow = autocorr(IncrementsLow1, 100)                 # ... and the autocorrelation

<div class="question">


    
**Q.** What is the size of the output variable `ACFLow`? How does this size correspond to the lags?




    
**A.** `ACFLow` is a vector with dimensions 1 $\times$ 101. The 101 values correspond to lag indices 0 to 100 (or lag times 0 ms to 100 ms).



</div>

In order to examie history dependence going back 100 ms, we need 100 lags (because each lag index corresponds to 1 ms). There are now too many values to examine them printed one by one at the command line, so instead we construct a plot fo the autocorrelation function with lag on the $x$-axis and correlation on the $y$-axis. Let's also include in this figure two approximate significance lines at $\pm2/\sqrt N$.

In [ ]:
plot(ACFLow, '.')        # Plot autocorrelation vs lags,
N1 = len(IncrementsLow1)                    # ... compute the sample size
sig = 2 / np.sqrt(N1)                       # ... and the significance level
plot([0, 100], [sig, sig], 'r:')           # ... and plot the upper and lower significance lines
plot([0, 100], [-sig, -sig], 'r:')
plt.xlim([0, 100])                          # ... set x-limits
plt.ylim([-.1, .1])                         # ... and y-limits
show()

We see that, the two approximate significance lines at $\pm2/\sqrt N$ suggest significant negative correlation structure is present up to about $6$ ms. This reflects the refractory period of the neuron: if you observed a spike in the previous 6 ms, you are less likely to observe a spike in the next few milliseconds. Beyond this point, the values of the autocorrelation mostly remain between the two significance bounds.

<div class="math-note">

**Alert!** We are using these significance bounds here for exploratory purposes. We are not performing a rigorous statistical test for significance of the autocorrelation at every lag.

</div>

If we choose to perform a rigorous statistical test for the significance of the autocorrelation, we would face the multiple comparisons problem. Briefly, if we perform many independent tests, and each has a 5% chance of reaching significance by chance, then the probability that any of these tests reaches significance by chance can be very large. If we wanted to perform many tests, we would need to control for multiple comparisons by adjusting the significance level so that the probability of any test being significant by chance is small. In the plot above, the significance lines are not corrected for multiple comparisons. Therefore, we accept that some of the correlation values that exceed these bounds may occur by chance. However, it is still very unlikely that all the significant correlations we observed from 1 to 6 ms are occurring purely by chance.

Now that we’ve computed and interpreted the autocorrelation function for the low-light condition, let’s compare it to the autocorrelation in the high-light condition. We repeat our previous commands using the `SpikesHigh` data values and choosing a time bin of 1 ms:
<a id=fig:8-12></a>

In [ ]:
IncrementsHigh1, _ = np.histogram(SpikesHigh, time_bins) # Compute the histogram to create increment process
ACFHigh = autocorr(IncrementsHigh1, 100)                 # ... and the autocorrelation
plot(ACFHigh, '.')                                       # Plot the autocorrelation,
sig = 2 / np.sqrt(len(IncrementsHigh1))                  # ... compute and plot the significance level,
plot([0, 100], [sig, sig], 'r:')                               
plot([0, 100], [-sig, -sig], 'r:')
plt.xlim([0, 100])                                       # ... and set the plot limits
plt.ylim([-.1, .1])
xlabel('Time [ms]')
ylabel('Autocorrelation')
show()

<div class="question">


    
**Q.** Consider the autocorrelation of the spike train data in the high-light condition shown above. What do you observe? How do the autocorrelations differ in the two conditions?

**A.** We find in the high-light condition significant correlation structure going all the way out to about 50 ms. Once again, we see refractoriness reflected in the negative autocorrelation at a lag of 1 ms, but this lasts much less time than in the low-light condition. Instead, there is now significant positive correlation at intermediate lags (approximately 2–50 ms). This positive correlation at short time lags reflects the tendency of the neuron to fire in bursts with small ISIs in the high-light condition; after a spike, another spike is more likely to occur in the next 2–50 ms than in the subsequent 50–100 ms.
</div>

Now that we’ve visualized the autocorrelations in the two light conditions, we can ask an important related question: Are the differences in the autocorrelations between these two conditions real? To answer this, we compute the difference in the autocorrelation functions between the low- and high-light conditions at every lag. If we assume that the firing in each condition is independent, the significance bounds for this difference can be computed by adding the variance of the autocorrelation from each condition. The standard deviation of the autocorrelation for the low-light condition is $1/\sqrt{N_1}$, so the variance of the auto- correlation for the low-light condition is $1/N_1$. For the high-light condition, the variance of the autocorrelation is $1/N_2$. We plot the differenced autocorrelations and the significance bounds:

In [ ]:
N2 = len(IncrementsHigh1)
ACFDiff = ACFHigh - ACFLow                    # Compute differences of autocorrelations
plot(ACFDiff, '.')                            # ... and plot them
sd = np.sqrt(1/N1+1/N2)                      # ... with significance bounds
plot([0, 100], [2 * sd * x for x in [1, 1]], 'r:')
plot([0, 100], [-2 * sd * x for x in [1, 1]], 'r:')
plt.xlim([0, 100])                            # Set the plot limits and label the axes
plt.ylim([-.1, .1])
xlabel('Time [ms]')
ylabel('Autocorrelation difference')
show()

The results suggest significant differences in the autocorrelation between the two conditions at intermediate time lags (at approximately 2–50 ms). These are the same time lags we identified with bursting activity in the high-light condition. This suggests that the neuron fires with more intermediate ISIs in the bursting range in the high-light condition.

[Back to top](#top)
<a id="acISI"></a>
### Computing Autocorrelations of the ISIs

The autocorrelation of the increments indicates the amount of time for which there are dependencies in the spiking data. In the high-light condition, we found large correlation values extending out to approximately 50 ms. This could be a consequence of the influence of patterns of many spikes with shorter ISIs or of single spikes with longer ISIs. We can distinguish between these possibilities by looking at the autocorrelation of the sequence of ISIs. In this case, the lag represents the number of spikes in the past rather than the amount of time in the past. If the dependence is only due to the last spike, we expect the ISIs to be uncorrelated at any nonzero lag. This would necessarily be true for data from a Poisson process. If we see correlation between ISIs, this suggests that the data do not come from a Poisson process and that the past spiking has an influence over multiple spikes. To investigate this, let’s compute the autocorrelation of the sequence of ISIs for the low-light condition:
<a id="fig:8-14"></a>

In [ ]:
# Compute and plot the autocorrelation of the low-light ISIs,
ISI_ACF_Low = autocorr(ISIsLow, 20)
plot(ISI_ACF_Low, '.')
# ... with upper and lower significance lines,
N3 = len(ISIsLow)
sd = 1 / np.sqrt(N3)
plot(2 * sd * np.ones_like(ISI_ACF_Low), 'r:')
plot(-2 * sd * np.ones_like(ISI_ACF_Low), 'r:')
# Set plot limits and label axes
plt.xlim([0, 20])
plt.ylim([-.2, .2])
xlabel('Lags')
ylabel('Autocorrelation')
show()

Notice that we include confidence bounds determined by the size of the sequence of interest (in this case, the length of the ISIs).
We see that the autocorrelation function has just a few isolated lags that are outside of the significance bounds. This could indicate a weak relation at particular lags or could be due to chance. Assuming the latter suggests that the data may come from a renewal process, a spiking process with independent ISIs for which the probability of a spike at any time only depends on the time of the most recent spike. One advantage of working with renewal processes is that it is fairly easy to write down and fit statistical models to the data. That is our next step.

<div class="question">

**Q.** Compute the correlation between ISIs for the data in the high-light condition. What do you find? Are these data consistent with a renewal process?

</div>

[Back to top](#top)
<a id="models"></a>
### Building Statistical Models of the ISIs

In the previous sections, we constructed autocorrelation functions of the increment processes and autocorrelation functions of the sequences of ISIs. The former suggested dependence going back up to $\approx$ 50 ms (<a href="#fig:8-12" class="fig">figure<span><img src="imgs/8-12.png"></span></a>), while the latter suggested that the spiking at any time depends only on the timing of the most recent spike (<a href="#fig:8-14" class="fig">figure<span><img src="imgs/8-14.png"></span></a>). We now consider another powerful technique to understand these data: building a model. More specifically, we construct a *statistical model* of these data. This model captures important features of the data but does not consist of explicit biophysical components (an example of a biologically explicit model is the Hodgkin-Huxley equations [[Hodgkin & Huxley, 1952](https://doi.org/10.1113/jphysiol.1952.sp004764)]. The notion of a model can be confusing and is audience dependent, so we clarify here.

To construct a statistical model for these data we assume that the ISIs are independent samples from some unknown distribution. We typically posit some class of distributions from which the data might arise, and identify the one distribution in that class that maximizes the chance of observing the actual data.

What class of distributions should we use to build an ISI model? Previously, we discussed a Poisson process as a basic model for a spiking process, consistent with the conceptual idea of spikes as coin flips. Let’s fit a Poisson process with a constant firing rate to the observed data. In other words, we begin with a model where the number of spikes in any time bin is independent of all previous (and future) spiking and has a Poisson distribution with a fixed but unknown rate parameter $\lambda$. The probability $P$ of $k$ spikes in any time bin is given by the Poisson distribution,
<a id="eq:6"></a>

$$
P(k) = \frac{
\lambda^k e^{-\lambda}
}{
k!
},
\tag{6}
$$

where $k!$ is the factorial of $k$. Under this model, the distribution for the number of spikes in a bin is Poisson, but what is the distribution of the waiting time between spikes (i.e., what is the distribution of the ISIs)? It can be shown that for any Poisson process with constant firing rate the ISIs have an exponential distribution [[Kass, Eden & Brown, 2014](http://dx.doi.org/10.1007/978-1-4614-9602-1)]. Mathematically, the probability density function for any ISI taking on a value $x$ is
<a id="eq:7"></a>

$$
f(x) = \lambda \exp(-\lambda x),
\tag{7}
$$

where $\lambda$ is the rate parameter for the Poisson process.

<div class="math-note">

**Alert!** This is a common point of confusion. The increments of a Poisson process have a Poisson distribution, and the ISIs have an exponential distribution. The Poisson distribution takes on non-negative integer values {0,1,...,$\infty$}, which make it appropriate for counting the number of spikes in an interval. The Poisson distribu- tion does not make sense to describe the waiting time between spikes, since this typically takes on a continuous value in [0, $\infty$].

</div>

Our goal is to find a good value of $\lambda$ so that our statistical model (<a href="#eq:7" class="thumb">7<span><img src="imgs/eq7.png"></span></a>) matches the observed ISI distributions. Let’s guess some values for $\lambda$, evaluate the model (<a href="#eq:7" class="thumb">7<span><img src="imgs/eq7.png"></span></a>), and see how well the model matches the data. We plot the probability of observing ISI values in 1 ms bins for the low-light condition. This is similar to the ISI histogram we plotted previously except that the $y$-axis should represent probability instead of counts. To do so, we simply divide each count value by the total number of ISIs in the low-light condition:

In [ ]:
bins = np.arange(0, .5, 0.001)           # Define 1 ms bins for histogram,
counts, _ = np.histogram(ISIsLow, bins)  # ... compute histogram of the ISIs,
prob = counts / len(ISIsLow)             # ... convert to probability,
fig, ax = plt.subplots()                 # ... create figure and axes objects that we can reuse later,
ax.stem(bins[:-1], prob)                 # ... and plot the probabilities,
ax.set_xlim([0, 0.15])                   # ... with fixed x-limits,
xlabel('ISI [s]')                        # ... and axes labeled.
ylabel('Probability')
show()

Now, on the same figure, let's choose a value for $\lambda$ and plot the statistical model (<a href="#eq:7" class="thumb">7<span><img src="imgs/eq7.png"></span></a>):

In [ ]:
l = 5                                    # Choose a value for lambda,
model = l * np.exp(-l * bins) * 0.001    # ... and create the model,
ax.plot(bins, model, 'g')                # ... and plot the model in green
fig

In this code, we have chosen $\lambda$ = 5 Hz and evaluated the statistical model at each time bin. We’ve also scaled the statistical model by a factor of 0.001 to match the 1 ms bin size, and plotted the model on top of the empirical ISI probability distribution. 

<div class="question">

**Q.** Try using some different values of $\lambda$. What values of $\lambda$ appear to provide a good fit to the empirical distribution of ISI values?

</div>



The process of guessing values of $\lambda$ and comparing the model (<a href="#eq:7" class="thumb">7<span><img src="imgs/eq7.png"></span></a>) to the empirical ISI distribution is not satisfying. How do we identify the parameter $\lambda$ that best fits the observed ISI distribution? We now consider a procedure to do so. Our goal is to find the value of $\lambda$ that maximizes the likelihood of the data given the statistical model (<a href="#eq:7" class="thumb">7<span><img src="imgs/eq7.png"></span></a>); this value of $\lambda$ will be the best fit of the model to the data. To implement this procedure, let’s consider the probability density of observing a sequence of ISIs, $x_1, x_2, ..., x_n$. If we assume that the ISIs are independent, then the probability density is
<a id="eq:8"></a>

$$
\begin{eqnarray}
f(x_1, x_2, ..., x_n) &=& f(x_1) f(x_2) ... f(x_n) \\ 
&=& \lambda \exp(-\lambda x_1) \lambda \exp(-\lambda x_2) ... \lambda \exp(-\lambda x_n) \\
&=& \lambda ^n \exp(-\lambda \sum_{i=1}^n x_i).
\end{eqnarray}
\tag{8}
$$

We call this expression the joint probability distribution of the observed data. In the first equality, we separate the joint probability distribution $f(x_1,x_2,...,x_n)$ into a product of probability distributions of each event (i.e., $f(x_1)$, the probability of the first ISI equaling $x_1$ , multiplied by $f(x_2)$, the probability of the second ISI equaling $x_2$, multiplied by $f(x_3)$, the probability of the third ISI equaling $x_3$, and so on). This partitioning of the joint probability is valid here because we assume the ISIs are independent. In the second equality, we replace each probability distribution with the exponential distribution we expect for the ISIs of a Poisson process. In the last equality, we rewrite the expression as a single exponential. Notice that this last expression is a function of the unknown rate parameter, $\lambda$.

When considered as a function of the unknown parameters, the joint distribution of the data (<a href="#eq:8" class="thumb">8<span><img src="imgs/eq8.png"></span></a>) is also called the *likelihood*. In this case, we write
<a id="eq:9"></a>

$$
L(\lambda) = \lambda^n e^{-\lambda (x_1 + x_2 + ... + x_n)},
\tag{9}
$$

to indicate that the likelihood $L$ is a function of $\lambda$. To understand what the likelihood function $L(\lambda)$ looks like , let's plot it. We do so for the data from the low-ight condition, and consider a range of possible $\lambda$ values.
<a id="fig:8-16"></a>

In [ ]:
lambdas = np.arange(50)  # Range of lambda values.
N3 = len(ISIsLow)        # Number of low-light ISIs observed.
L = lambdas ** N3 * np.exp(-lambdas * sum(ISIsLow))  # Compute the likelihood,
plot(lambdas, L)         # ... and plot it
xlabel('$\lambda$')
ylabel('Likelihood')
show()

<div class="question">
    
**Q.** Consider the plot above. Does this answer seem okay?
  
**A.** Something went wrong here. The plot gives part of a line that rises toward $10^{-13}$ and is zero elsewhere. Why does this happen, and how can we fix it? To answer this, consider the first term in the likelihood function, $\lambda ^n$. In this case we are raising $\lambda$ to the power of `len(ISIsLow)` = 749. This is beyond the numerical precision limits of standard Python computations.

</div>

So, we can’t easily plot the likelihood directly. Instead, we plot the log of the likelihood. In this case, computing the log is useful because extremely large values are reduced to a more manageable range.
<a id="fig:8-17"></a>

In [ ]:
lambdas = lambdas[1:]  # Update the lambda range to exclude 0.
l = N3 * np.log(lambdas) - lambdas * sum(ISIsLow)  # Compute the log likelihood,
plot(lambdas, l)       # ... and plot it.
xlabel('$\lambda$')
ylabel('Log likelihood')
show()

<div class="question">

**Q.** Consider the second line of code above. Does the definition for `l` correspond to $\log[L(\lambda)]$ (<a href="#eq:9" class="thumb">eq.<span><img src="imgs/eq9.png"></span></a>)? *Hint*: It should. Remember $\log(x^a)=a \log x$, and $\log(e^b)=b$.

</div>

We see that the log likelihood is low for small $\lambda$, rises quickly as $\lambda$ increases, and then starts to fall off once $\lambda$ becomes larger than $\approx$ 25. The point $\lambda$ = 25, where the log likelihood is maximized, is called the maximum likelihood estimate of $\lambda$. We use the symbol $\hat\lambda$ to denote the maximum likelihood estimate of $\lambda$.

We observe that although the values of the likelihood go beyond the precision range in Python, the peak in the log likelihood stands out very clearly. Note that the likelihood (<a href="#fig:8-16" class="fig">figure<span><img src="imgs/8-16.png"></span></a>) is maximized at the same point as the log likelihood (<a href="#fig:8-17" class="fig">figure<span><img src="imgs/8-17.png"></span></a>). This is always true.


<div class="question">

**Q.** Can you explain why?

</div>

We could also have computed the maximum likelihood estimator theoretically, by differentiating the log likelihood with respect to $\lambda$, setting that equal to zero, and solving for $\lambda$. This gives $\frac{n}{\hat\lambda} - \sum_{i=1}^n x_i = 0$, which can be solved to find $\hat\lambda=(\sum_{i=1}^n x_i)^{-1} = 1 / \hat x = 25.0$ spikes/s. Remember that $x_i$ is the $i^{th}$ ISI value, so $\bar x$ is the average ISI value. This computation shows that the maximum likelihood estimate for the rate parameter of a Poisson process is just 1 divided by the average ISI value. For some statistical models, it is convenient to compute maximum likelihood estimates theoretically in this manner, but sometimes no closed-form solution exists. In these cases, we typically use numerical methods to solve for the maximum likelihood estimates. 

<div class=question>
    
**Q.** What is the maximum likelihood estimate for the Poisson rate parameter in the high-light condition?

    
**A.** Repeating the analysis for the high-light condition, the maximum likelihood estimate for a Poisson rate parameter is $\hat \lambda = n(\sum{i=1}^nx_i)^{-1} = 1 / \bar x = 32.3$ spikes/s. The differenc ein the Poisson rate parameter of 32.3 - 25.0 = 7.3 spikes/s reflects the difference in the overall firing rate of the neuron between the low-and high-light conditions.
    
</div>

<div class=question>

**Q.** Is the difference in the Poisson rate parameter between the low-and high-light conditions statistically significant?

</div>

To address this last question, let’s use a bootstrap analysis (see [chapter 2](../2.%20The%20Event-Related%20Potential/The%20Event-Related%20Potential.ipynb#bootstrap)). We combine all the ISIs from both conditions into one pool, sample many new datasets with replacement from that pool, and compare the actual difference in rate parameters to the distribution of differences across the samples.

In [ ]:
# Compute the observed difference in lambdas
MLDiff = 1 / ISIsHigh.mean() - 1 / ISIsLow.mean()

# Then, perform the bootstrap analysis.
ISIs = np.hstack([ISIsLow, ISIsHigh])  # Merge all ISIs.
Nall = len(ISIs)  # Save length of all ISIs.
Nlo = len(ISIsLow)  # Save length for the low-light condition
Nhi = len(ISIsHigh)  # Save length high-light condition

# Compute the difference in lambdas from resampled data
sampDiff = [1 / np.mean(ISIs[np.random.randint(Nall, size=Nhi)]) -  # Resample the high-light ISIs and subtract
           1 / np.mean(ISIs[np.random.randint(Nall, size=Nlo)])     # ... the resampled low-light ISIs
           for _ in range(1000)]                                    # ... 1000 times

# Compare the bootstrap distribution to the empirical
plt.hist(sampDiff, bins=30)       # Plot resampled ISIs distribution
plot([MLDiff, MLDiff], [0, 100])  # ... and the empirical ISIs.
xlabel('Counts')
ylabel('Difference in rate (spikes/s)')
show()

<div class=math-note>

There are more powerful tests we could use to compare the Poisson rate parameters. By more powerful, we mean that the tests are more likely to show a significant difference when one is actually present. However, the fact that the bootstrap test gives a significant result suggests that these more powerful tests would also be significant.

</div>

But, is the Poisson model good? To answer this, let’s visualize the model fits compared to the data. There are a number of ways to do this. We start by comparing the expected proportion of ISIs for a Poisson process to the ISI histograms we actually observe in each condition. Let’s do so first for the low-light condition:

In [ ]:
bins = np.arange(0, .5, 0.001)            # Define 1 ms bins for histogram
counts, _ = np.histogram(ISIsLow, bins)   # Compute histogram
prob = counts / len(ISIsLow)              # ... convert to probability,
plt.bar(bins[:-1], prob, width=0.001)     # ... and plot probability.
lbda = 1 / ISIsLow.mean()                 # Compute best guess for lambda,
model = lbda * np.exp(-lbda * bins) * 0.001  # ... build the model,
plot(bins, model, 'r')                    # ... and plot it
plt.xlim([0, 0.15])                       # ... xlim from 0 to 150 ms,
xlabel('ISI [s]')                         # ... label the x-axis,
ylabel('Probability')                     # ... and the y-axis
show()


<div class=question>

**Q.** Compare the model fit to the empirical ISI distribution for the low-light condition. Does the model fit the data?

**A.** No, the model does not provide a very good fit to the data. Since Poisson processes have spikes that are independent of past activity, they do not capture either the refractoriness (i.e., the few spikes observed at short times) or the bursting (i.e., the increased spiking at times 5–20 ms) that we observe in the data.

</div>

<div class=question>
    
**Q.** Repeat the analysis and compare the empirical ISI histogram to the best-fit model in the high-light condition. Does the model fit the data?
    
</div>

To go beyond visual inspection of the model fits and quantify the goodness of fit, we compare the cumulative distributions computed from the data and model. The *cumulative distribution function* (CDF), $F(x)$, is the probability that a random variable will take on a value less than or equal to $x$. For the exponential ISI model with rate parameter $\lambda$, the model CDF is

$$
\begin{eqnarray}
F_{mod}(x) &=& \Pr(\mbox{ISI} \leq x) \\
&=& \int_0^x \lambda e^{-\lambda t} dt \\
&=& 1 - e^{-\lambda x}.
\end{eqnarray}
$$

We compare this to the empirical CDF of the data, $F_{emp}(x)$, which is defined as the proportion of observations less than or equal to $x$. The code to compute and plot these CDFs for the low light-condition is as follows:

In [ ]:
bins = np.arange(0, 0.5, 0.001)     # Define 1 ms bins for histogram
lbda = 1 / ISIsLow.mean()           # Compute best guess for lbda,
FmodLow = 1 - np.exp(-lbda * bins)  # ... and define model CDF.
FempLow = np.cumsum(prob)           # Define empirical CDF
plot(bins, FmodLow)                 # Plot the model CDF,
plot(bins[:-1], FempLow, 'r')       # ... and the empirical CDF,
plt.xlim([0, 0.2])                  # ... with specified x-limits
xlabel('Time [s]')
ylabel('CDF')
show()

<div class=question>
    
**Q.** Have you used the function `cumsum()` before? If not, look it up using `np.cumsum?`

</div>

<div class=question>

**Q.** Compare the model and empirical CDFs in the plot above. What do you think?

**A.** If the model were a perfect fit, the red and blue curves would align. However, that's not what we find here. We conclude that the model may not provide a good fit to the data.

</div>

<div class=question>

**Q.** Compare the model and empirical CDF for the data in the high-light condition. What do you find? Is the model a good fit to the data?

</div>

Another common way to visualize the difference between the model and empirical distributions is a *Kolmogorov-Smirnov* (KS) plot. This is just a plot of the empirical CDF against the model CDF directly.

In [ ]:
fig, ax = plt.subplots()
plot(FmodLow[:-1], FempLow)    # Plot the model vs empirical CDFs.
plt.axis([0, 1, 0, 1])         # Set the axes ranges.
xlabel('Model CDF')
ylabel('Empirical CDF')
show()

Since the KS plot compares CDFs, both the $x$-axis and $y$-axis range from 0 to 1. A perfect fit between the model and empirical CDFs would look like a straight, 45-degree line between the points (0,0) and (1,1). Any deviation from this line represents deviation between the observed and model distributions. One nice result for comparing CDFs is that with enough data, the maximum difference between the model and empirical CDFs has a known asymptotic distribution, which can be used to put confidence bounds about the KS plot [[Kass, Eden & Brown, 2014](http://dx.doi.org/10.1007/978-1-4614-9602-1)]. For 95% confidence bounds, a well-fit model should stay within ±1.36/$\sqrt N$ of the 45-degree line, where $N$ is the number of ISIs observed. Let’s place these confidence bounds on the KS plot:

In [ ]:
Nlow = len(ISIsLow)  # Length of the low-light condition
# Plot the confidence bounds
ax.plot([0, 1], [x + 1.36 / np.sqrt(Nlow) for x in [0, 1]], 'k:')
ax.plot([0, 1], [x - 1.36 / np.sqrt(Nlow) for x in [0, 1]], 'k:')
fig

A well-fit model should stay entirely within these bounds. In this case, the KS plot for the low-light condition extends well outside these bounds. The exponential ISI model—or equivalently, the Poisson process model—does not fit the data in the low-light condition well. This suggests that we need a better model if we want to make meaningful comparisons about differences in the structure of the data between the two conditions.

<div class=question>

**Q.** Compute the KS plto with 95% significance bounds or the high-light condition. Does the exponential ISI model fit the data well?

</div>

#### A More Advanced Statistical Model.
We’ve now investigated one class of models, the exponential distribution, to fit the observed ISI distributions. However, through analysis, we’ve found that this statistical model is not sufficient to mimic the observed data. There are many other choices for statistical models; let’s try one other class of models. The inverse Gaussian probability model has already been used successfully to describe ISI structure in this system [Hodgkin & Huxley, 1952](https://doi.org/10.1007/BF02459568). The mathematical expression for the inverse Gaussian probability density is 

$$
f(x) = \sqrt{\frac{\lambda}{2 \pi x^3}}\exp\left(\frac{-\lambda(x - \mu)^2}{2 x \mu^2}\right)
\tag{10}
$$

The inverse Gaussian distribution has two parameters that determine its shape: $\mu$, which determines the mean of the distribution, and $\lambda$, which is called the shape parameter. At $x$ = 0, the inverse Gaussian has a probability density equal to zero, which suggests it could capture some of the refractoriness seen in the data.

If we again assume that the ISIs are independent of each other, then the likelihood of observing the sequence of ISIs, $x_1 , x_2 , . . . , x_n$, is the product of the probabilities of each ISI,

$$
L(\mu, \lambda) = f(x_1, x_2, ..., x_n) = \prod_{i=1}^N\sqrt{\frac{\lambda}{2\pi x_i^3}}\exp\left(\frac{-\lambda(x_i - \mu)^2}{2 x_i \mu^2}\right)
\tag{11}
$$

The log likelihood is then

$$
\log\big(L(\mu, \lambda)\big) = \frac{N}{2}\log{\lambda}{2\pi} - \frac{3}{2}\sum_{i=1}^N \log{x_i} - \sum_{i=1}^N\frac{\lambda(x_i - \mu)^2}{2x_i \mu^2}.
\tag{12}
$$

Since this distribution has two parameters, the maximum likelihood solution for this model is the pair of parameter estimates $\hat\mu$, $\hat\lambda$ that maximizes the likelihood of the data. We can solve for the maximum likelihood estimate analytically by taking the derivative with respect to both parameters, setting these equal to zero, and solving the resulting set of equations. In this case, the maximum likelihood estimators are

$$
\hat\mu = \frac{1}{N}\sum_{i=1}^N x_i
\tag{13}
$$

and

$$
\hat\lambda = \left( \frac{1}{N}\sum_{i=1}^N\left(\frac{1}{x_i} - \frac{1}{\bar \mu}\right)\right)^{-1}.
\tag{14}
$$

Using this expression, we can fit an inverse Gaussian model to the data in each condition and evaluate the goodness-of-fit of the model. Let’s do so now for the low-light condition.

In [ ]:
bins = np.arange(0, .5, 0.001)    # Define 1 ms bins.
Nlow = len(ISIsLow)               # Length of low-light condition.
mu = ISIsLow.mean()               # Mean of inverse Gaussian
lbda = 1 / (1 / ISIsLow - 1 / mu).mean()    # ... and shape parameter
model = (                                   # ... to create the model.
    np.sqrt(lbda / 2 / np.pi / bins ** 3) * 
    np.exp(-lbda * (bins - mu) ** 2 /       
           2 / mu ** 2 / bins) * 0.001
)
model[0] = 0                      # Numerator to 0 faster than denominator.
print('mu = ', mu)                # Display the MLEs
print('lambda = ', lbda)

<div class=python-note>

Note that the first element of `bins` is 0, so dividing by `bins` causes a divide by zero warning.

</div>

From the computations, we find maximum likelihood estimates $\mu$ = 40.0 ms and $\lambda$ = 49.3 ms in the low-light condition. Next, we plot the data and the model.

In [ ]:
# Plot the data and the model,
plt.subplot(121)
counts, _ = np.histogram(ISIsLow, bins)  # Compute histogram,
prob = counts / len(ISIsLow)             # ... convert to probability,
plt.bar(bins[:-1], prob, width=1e-3)     # ... and plot probability
plot(bins, model, 'r')                   # Plot the model
plt.xlim([0, 0.2])                       # xlim from 0 to 200 ms.
xlabel('ISI [s]')                        # Label the axes
ylabel('Probability')

# Plot the KS plot
plt.subplot(122)
FmodLow = np.cumsum(model[:-1])          # Define the model CDF,
FempLow = np.cumsum(prob)                # ... and define empirical CDF,
plot(FmodLow, FempLow)                   # ... plot model vs empirical CDF,
plot([0, 1], np.arange(2) + 1.36 / np.sqrt(Nlow))  # ... upper confidence bound,
plot([0, 1], np.arange(2) - 1.36 / np.sqrt(Nlow))  # ... lower confidence bound,
plt.axis([0, 1, 0, 1])                   # ... set the axes ranges,
xlabel('Model CDF')                      # ... and label the axes.
ylabel('Empirical CDF')
show()

<div class=question>

**Q.** Consider the fit of the inverse Gaussian model to the data in the low-light condition. Does the inverse Gaussian model provide a good fit to the ISIs?

**A.** This model provides a much better fit to the data; the KS plot is contained within the 95% confidence bounds.

</div>

<div class=question>

**Q.** Consider the fit of the inverse Gaussian model to the data in the high-light con- dition. Does the inverse Gaussian model provide a good fit to these ISIs?

</div>

<div class=question>

**Q.** Compare the estimates of the two parameters $\mu$ and $\lambda$ of the inverse Gaussian model in the two conditions. What do these reveal about the differences between the low- and high-light conditions?

</div>

[Back to top](#top)
<a id="summary"></a>
## Summary
In this chapter, we considered the spiking activity recorded in two conditions. We began with visualizations of the spiking data, and construction and visualization of the increment process (i.e., binned spike counts). We then assessed the variability in the increments through computation of the Fano factor, and showed that the low- and high-light conditions had less and more variability, respectively, than expected for a Poisson process. We also assessed the autocorrelation of the increment processes and observed the impact of refractoriness and bursting activity. In addition, we created and visualized the ISIs for each condition. Inspection of the ISI histograms suggested bursting activity in both conditions, and more small ISIs in the high-light condition. Analysis of the ISI autocorrelations revealed no compelling evidence for dependence and supported the hypothesis of a renewal process. Finally, we built two statistical models of the observed ISIs. We discussed how to fit the model parameters by computing the maximum likelihood estimate, and how to evaluate the model goodness-of-fit to the data using the KS plot. We showed that the first model—the Poisson process as a model of spiking with a corresponding exponential distribution of ISIs—did not fit the observed ISI data. A second model—the inverse Gaussian probability model—provided a much more accurate fit to the observed ISIs. The modeling suggests that at least two features of the spiking activity have changed from the low-light to the high-light condition. First, the mean ISI is smaller, and hence the average firing rate is larger, in the high-light condition. Second, the shape of the firing distribution has changed so that the cell is more likely to fire in bursts with short ISIs in the high-light condition.

[Back to Top](#top)
<a id="appendix"></a>
## Appendix: Spike Count Mean and Variance for a Poisson Process

In this appendix, we compute the theoretical mean $\mu$ and the theoretical variance of the spike count $\sigma^2$ for a Poisson process. Let’s compute $\mu$ using a general formula that makes use of the probability $P(k)$ of observing $k$ spikes,
<a id="eq:15"></a>

$$
\mu = \sum_{k=1}^\infty k P(k).
$$

Replacing $P(k)$ with the expression for a Poisson distribution 
(<a href="#eq:6" class="thumb">eq.<span><img src="imgs/eq6.png"></span></a>)
, we find

$$
\begin{eqnarray}
\mu &=& \sum_{k=0}^\infty k\left(\frac{\lambda^k e^{-k}}{k!}\right) \\
&=& e^{-\lambda}\sum_{k=0}^\infty k\frac{\lambda^k}{k!}.
\end{eqnarray}
$$

To make progress, let's write out the terms in the summation,
<a id="eq:16"></a>

$$
\begin{eqnarray}
\mu &=& e^{-\lambda}
\left( 
0 + 
\frac{\lambda^1}{1!} + 
2\frac{\lambda^2}{2!} + 
3\frac{\lambda^3}{3!} + 
4\frac{\lambda^4}{4!} + 
\cdots 
\right) \\
&=& e^{-\lambda}\lambda
\left(
1 + 
\frac{\lambda^2}{2!} + 
\frac{\lambda^3}{3!} + 
\cdots 
\right) \\
&=& e^{-\lambda}\lambda(e^\lambda) \\
&=& \lambda,
\end{eqnarray}
$$

where we have used the fact that $e^x = 1 + \frac{x^2}{2!} + \frac{x^3}{3!} + \frac{x^4}{4!} + \cdots$. Notice that the mean spike count equals the rate parameter of the Poisson process.

To find the spike count variance for a Poisson process, we follow a similar procedure. In general, we compute the variance $\sigma^2$ of the spike count $k$ with probability distribution $P(k)$ as 
<a id="eq:17"></a>

$$
\sigma^2 = \sum_{k=0}^\infty k^2 P(k) - \left(\sum_{k=0}^\infty k P(k)\right) ^2.
$$

As in our computation of the mean spike count, we replace $P(k)$ with (<a href=#eq:6 class=thumb>eq.<span><img src=imgs/eq6.png></span></a>)
, the expression for a Poisson process. Notice that the second term is the square of the expression (<a href=#eq:15 class=thumb>eq.<span><img src=imgs/eq15.png></span></a>)
, and for a Poisson process we found $\mu = \lambda$. Therefore, let’s replace the second term in (<a href=#eq:17 class=thumb>eq.<span><img src=imgs/eq17.png></span></a>)
 with $\sigma^2$ and substitute for $P(k)$ in the first term of (<a href=#eq:17 class=thumb>eq.<span><img src=imgs/eq17.png></span></a>) to find
 
$$
\begin{eqnarray}
\sigma^2 &=&
\sum_{k=0}^\infty
k^2
\left(
\frac{\lambda^k e^{-\lambda}}{k!}
\right)
- \lambda^2
\\
&=&
e^{-\lambda}
\sum_{k=0}^\infty
k^2
\frac{\lambda^k}{k!} 
- \lambda^2.
\end{eqnarray}
$$

To make progress, we follow the same strategy and write out the terms in the summation,

$$
\begin{eqnarray}
\sigma^2
&=&
e^{-\lambda}
\left(
0 + 
\lambda +
2^2\frac{\lambda^2}{2!} + 
3^2\frac{\lambda^3}{3!} + 
4^2\frac{\lambda^4}{4!} + 
\cdots
\right) - 
\lambda^2
\\
&=&
\lambda e ^{-\lambda}
\left(
1 + 
2\lambda + 
\frac{3}{2}\lambda ^2 + 
\frac{4}{6}\lambda ^2 + 
\cdots
\right) - 
\lambda^2.
\end{eqnarray}
$$

Now, we divide this sum of terms into two pieces, a "nice term" (in the first brackets) and "leftovers" (in the second brackets):

$$
\sigma^2 = \lambda e^{-\lambda}
\left(
\left[
1 + 
\lambda + 
\frac{\lambda^2}{2!} + 
\frac{\lambda^3}{3!} + 
\cdots
\right] + 
\left[
\lambda + 
\frac{2\lambda^2}{2!} + 
\frac{3\lambda^3}{3!} + 
\cdots
\right] 
\right) - 
\lambda^2.
$$

We can simplify by recognizing that 
$\left[
1 + 
\lambda + 
\frac{\lambda^2}{2!} + 
\frac{\lambda^3}{3!} + 
\cdots
\right]
=
e^\lambda$.
Then
<a id="eq:18"></a>

$$
\begin{eqnarray}
\sigma^2
&=&
\lambda e^{-\lambda}
\left(
e^\lambda + 
\lambda
\left[
1 + 
\lambda + 
\frac{\lambda^1}{1!} + 
\frac{\lambda^2}{2!} + 
\cdots
\right] 
\right)-
\lambda^2 \\
&=&
\lambda e^{-\lambda}
\left(
e^\lambda + \lambda e^\lambda
\right)
-\lambda^2 \\
&=&
\lambda + \lambda^2 - \lambda^2 \\
&=&
\lambda,
\end{eqnarray}
$$

where again we've used the definition of $e^\lambda$. We conclude that the spike count variance for a Poisson precess equals the firing rate $\lambda$.

Combining these results for the mean spike count $\mu$ 
(<a href=#eq:16 class=thumb>eq.<span><img src=imgs/eq16.png></span></a>)
and the spike count variance
(<a href=#eq:18 class=thumb>eq.<span><img src=imgs/eq18.png></span></a>),
we conclude that for a Poisson process,

$$\mu = \sigma^2 = \lambda,$$

and therefore for a Poisson process, the Fano factor $\sigma^2/\mu=1$.

[Back to top](#top)

In [ ]:
from IPython.core.display import HTML
HTML('../../assets/custom/custom.css')
# NO CODE